## Введение

В этом уроке мы рассмотрим:
- Что такое вызов функции и где он применяется
- Как создать вызов функции с помощью OpenAI
- Как интегрировать вызов функции в приложение

## Цели обучения

После завершения этого урока вы будете знать и понимать:

- Для чего используется вызов функции
- Как настроить вызов функции с помощью сервиса OpenAI
- Как разрабатывать эффективные вызовы функций для вашего приложения


## Понимание вызова функций

В этом уроке мы хотим создать функцию для нашего образовательного стартапа, которая позволит пользователям с помощью чат-бота находить технические курсы. Мы будем рекомендовать курсы, подходящие их уровню знаний, текущей должности и интересующим технологиям.

Для этого мы используем комбинацию:
 - `OpenAI` для создания чата с пользователем
 - `Microsoft Learn Catalog API` для поиска курсов по запросу пользователя
 - `Function Calling` для передачи запроса пользователя в функцию, которая выполнит API-запрос.

Для начала давайте разберёмся, зачем вообще использовать вызов функций:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # получаем новый ответ от GPT, который уже видит ответ функции


print(second_response.choices[0].message)


### Зачем нужен Function Calling

Если вы уже проходили другие уроки этого курса, вы, вероятно, понимаете, насколько мощными могут быть большие языковые модели (LLM). Надеюсь, вы также заметили и их ограничения.

Function Calling — это функция в OpenAI Service, созданная для решения следующих проблем:

Несогласованный формат ответов:
- До появления function calling ответы от языковой модели были неструктурированными и непоследовательными. Разработчикам приходилось писать сложный код для проверки и обработки всех возможных вариантов вывода.

Ограниченная интеграция с внешними данными:
- Раньше было сложно использовать данные из других частей приложения в чате с моделью.

Благодаря стандартизации формата ответов и возможности легко интегрировать внешние данные, function calling упрощает разработку и уменьшает необходимость в дополнительной проверке данных.

Пользователи не могли получить ответы вроде «Какая сейчас погода в Стокгольме?». Это связано с тем, что модели ограничены временем, на котором они были обучены.

Давайте рассмотрим пример ниже, который иллюстрирует эту проблему:

Допустим, мы хотим создать базу данных студентов, чтобы предлагать им подходящие курсы. Ниже приведены два описания студентов, которые очень похожи по содержанию данных.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

Мы хотим отправить это LLM для разбора данных. В дальнейшем это можно будет использовать в нашем приложении для отправки в API или хранения в базе данных.

Давайте создадим два одинаковых запроса, в которых мы укажем LLM, какая информация нас интересует:


Мы хотим отправить это LLM для разбора частей, важных для нашего продукта. Таким образом, мы можем создать два идентичных запроса, чтобы дать инструкции LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


После создания этих двух подсказок мы отправим их в LLM с помощью `openai.ChatCompletion`. Мы сохраняем подсказку в переменной `messages` и назначаем роль `user`. Это делается для имитации сообщения от пользователя, написанного в чат-бот.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Хотя подсказки одинаковые, а описания похожи, мы можем получить разные форматы свойства `Grades`.

Если выполнить ячейку выше несколько раз, формат может быть как `3.7`, так и `3.7 GPA`.

Это происходит потому, что LLM принимает неструктурированные данные в виде текстовой подсказки и также возвращает неструктурированные данные. Нам нужен структурированный формат, чтобы мы знали, чего ожидать при сохранении или использовании этих данных.

Используя функциональные вызовы, мы можем быть уверены, что получаем обратно структурированные данные. При использовании функциональных вызовов LLM на самом деле не вызывает и не выполняет никакие функции. Вместо этого мы создаём структуру, которой LLM должен следовать в своих ответах. Затем мы используем эти структурированные ответы, чтобы понять, какую функцию запускать в наших приложениях.


![Диаграмма потока вызова функции](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.ru.png)


### Примеры использования вызова функций

**Вызов внешних инструментов**  
Чат-боты отлично отвечают на вопросы пользователей. С помощью вызова функций чат-боты могут использовать сообщения пользователей для выполнения определённых задач. Например, студент может попросить чат-бота: "Отправь письмо моему преподавателю с просьбой о дополнительной помощи по этому предмету". Для этого можно вызвать функцию `send_email(to: string, body: string)`

**Создание API или запросов к базе данных**  
Пользователи могут находить информацию, используя естественный язык, который преобразуется в форматированный запрос или API-запрос. Например, учитель может спросить: "Кто из студентов сдал последнее задание?", что может вызвать функцию с именем `get_completed(student_name: string, assignment: int, current_status: string)`

**Создание структурированных данных**  
Пользователи могут взять текстовый блок или CSV-файл и использовать LLM для извлечения из него важной информации. Например, студент может преобразовать статью из Википедии о мирных соглашениях, чтобы создать AI-карточки для запоминания. Это можно сделать с помощью функции `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Создание первого вызова функции

Процесс создания вызова функции включает 3 основных шага:
1. Вызовите Chat Completions API с перечнем ваших функций и сообщением пользователя
2. Прочитайте ответ модели, чтобы выполнить действие, например, вызвать функцию или API
3. Сделайте еще один вызов к Chat Completions API с ответом вашей функции, чтобы использовать эту информацию для создания ответа пользователю.


![Flow of a Function Call](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.ru.png)


### Элементы вызова функции

#### Ввод пользователя

Первым шагом является создание сообщения пользователя. Это сообщение можно задать динамически, получив значение из текстового поля, или указать его здесь вручную. Если вы впервые работаете с API Chat Completions, необходимо определить `role` и `content` сообщения.

`role` может быть либо `system` (создание правил), `assistant` (модель), либо `user` (конечный пользователь). Для вызова функции мы укажем роль как `user` и приведём пример вопроса.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Создание функций.

Далее мы определим функцию и её параметры. Здесь мы будем использовать только одну функцию под названием `search_courses`, но вы можете создать несколько функций.

**Важно**: Функции включаются в системное сообщение для LLM и учитываются в общем количестве доступных вам токенов.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Определения**

Структура определения функции имеет несколько уровней, у каждого из которых есть свои свойства. Вот разбор вложенной структуры:

**Свойства функции верхнего уровня:**

`name` - Имя функции, которую мы хотим вызвать.

`description` - Описание того, как работает функция. Здесь важно быть конкретным и понятным.

`parameters` - Список значений и формат, которые вы хотите получить в ответе от модели.

**Свойства объекта parameters:**

`type` - Тип данных объекта параметров (обычно "object")

`properties` - Список конкретных значений, которые модель будет использовать в своем ответе

**Свойства отдельного параметра:**

`name` - Неявно определяется по ключу свойства (например, "role", "product", "level")

`type` - Тип данных этого конкретного параметра (например, "string", "number", "boolean")

`description` - Описание конкретного параметра

**Необязательные свойства:**

`required` - Массив, в котором перечислены параметры, обязательные для выполнения вызова функции


### Вызов функции
После того как функция определена, теперь нужно включить её в запрос к Chat Completion API. Для этого мы добавляем `functions` в запрос. В данном случае это `functions=functions`.

Также есть возможность установить параметр `function_call` в значение `auto`. Это значит, что мы позволим LLM самостоятельно определить, какую функцию вызвать, исходя из сообщения пользователя, вместо того чтобы указывать это вручную.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Теперь давайте посмотрим на ответ и обратим внимание на его форматирование:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Вы можете увидеть, что вызывается функция с определённым именем, и из сообщения пользователя LLM смог найти данные для заполнения аргументов этой функции.


## 3. Интеграция вызовов функций в приложение.

После того как мы протестировали отформатированный ответ от LLM, теперь мы можем интегрировать это в приложение.

### Управление процессом

Чтобы интегрировать это в наше приложение, давайте выполним следующие шаги:

Сначала вызовем сервисы OpenAI и сохраним сообщение в переменную с именем `response_message`.


In [ ]:
response_message = response.choices[0].message

Теперь мы определим функцию, которая будет вызывать API Microsoft Learn для получения списка курсов:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



В качестве лучшей практики мы сначала проверим, хочет ли модель вызвать функцию. После этого мы создадим одну из доступных функций и сопоставим её с вызываемой функцией.
Затем мы возьмём аргументы функции и сопоставим их с аргументами из LLM.

Наконец, мы добавим сообщение о вызове функции и значения, которые были возвращены сообщением `search_courses`. Это даст LLM всю необходимую информацию для ответа пользователю на естественном языке.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Задание по программированию

Отличная работа! Чтобы продолжить изучение вызова функций OpenAI, вы можете создать: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Больше параметров функции, которые могут помочь учащимся находить больше курсов. Доступные параметры API можно найти здесь:
 - Создайте еще один вызов функции, который принимает больше информации от учащегося, например, его родной язык
 - Реализуйте обработку ошибок на случай, если вызов функции и/или API не возвращает подходящих курсов



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные толкования, возникшие в результате использования данного перевода.
